# <font color="#418FDE" size="6.5" uppercase>**Task Specific Prompts**</font>

>Last update: 20260118.
    
By the end of this Lecture, you will be able to:
- Design prompts optimized for summarization, classification, and Q&A with Llama 3. 
- Implement LangChain chains that encapsulate task-specific prompts and Llama 3 calls. 
- Evaluate and iterate on prompts based on qualitative and simple quantitative feedback. 


## **1. Effective Summarization Prompts**

### **1.1. Controlling Summary Length**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_02/Lecture_B/image_01_01.jpg?v=1768764756" width="250">



>* Specify clear target length for summaries
>* Concrete length helps balance brevity and detail

>* Match summary length to purpose and detail
>* Explain use case so model prioritizes information

>* Adjust summary length iteratively based on outputs
>* Tailor length instructions to different audience needs



### **1.2. Key Point Extraction**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_02/Lecture_B/image_01_02.jpg?v=1768764776" width="250">



>* Ask for core ideas, not short summaries
>* Guide model to decisions, changes, key outcomes

>* Importance depends on reader role and goal
>* Role-based instructions yield targeted, useful key points

>* Aim for mid-level, actionable key point detail
>* Balanced granularity supports decisions and quick understanding



### **1.3. Robust Summaries From Noise**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_02/Lecture_B/image_01_03.jpg?v=1768764791" width="250">



>* Tell the model what matters and not
>* Have it ignore chatter, duplicates, and formatting

>* Flag contradictions, uncertainty, and missing information clearly
>* Report multiple views, separating facts from opinions

>* Tell the model which noisy parts to ignore
>* Prioritize real content so summaries stay accurate



## **2. Robust Classification Prompts**

### **2.1. Defining Classification Labels**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_02/Lecture_B/image_02_01.jpg?v=1768764812" width="250">



>* Define few, clear, non-overlapping classification labels
>* Base labels on observable text, not intent

>* Labels act as a contract driving actions
>* Define labels precisely to ensure consistent classification

>* Define labels for edge cases and ambiguity
>* Consistent definitions stabilize outputs and downstream decisions



### **2.2. Designing Few Shot Labels**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_02/Lecture_B/image_02_02.jpg?v=1768764827" width="250">



>* Use few-shot examples to teach classification behavior
>* Embed examples in chains to guide ambiguous inputs

>* Pick clear, realistic, diverse examples per label
>* Embedded examples act like a portable training set

>* Continuously update examples based on misclassifications
>* Iterative refinement makes the chain robust, aligned



In [ ]:
#@title Python Code - Designing Few Shot Labels

# Demonstrate few shot labels for simple message classification chain.
# Show how examples guide model style and label predictions.
# Use a tiny rule based mock model instead of real Llama three.
# pip install langchain-openai langchain-community.

# Define few shot examples for customer support classification labels.
few_shot_examples = [
    {"message": "My bill this month is higher than expected.", "label": "billing"},
    {"message": "Why was my credit card charged twice yesterday night?", "label": "billing"},
    {"message": "The app crashes whenever I try uploading photos.", "label": "technical_issue"},
    {"message": "Videos keep buffering on my home WiFi connection.", "label": "technical_issue"},
    {"message": "I cannot log into my account from my laptop.", "label": "account_access"},
]

# Define a simple rule based classifier using few shot patterns.
def classify_message(message, examples):
    message_lower = message.lower()
    billing_keywords = ["bill", "charged", "invoice", "payment"]
    technical_keywords = ["crash", "error", "bug", "buffer", "wifi"]
    access_keywords = ["login", "log in", "password", "account"]

    # Check billing related keywords using simple containment checks.
    if any(word in message_lower for word in billing_keywords):
        return "billing"

    # Check technical issue related keywords using simple containment checks.
    if any(word in message_lower for word in technical_keywords):
        return "technical_issue"

    # Check account access related keywords using simple containment checks.
    if any(word in message_lower for word in access_keywords):
        return "account_access"

    # Fall back to most common label from few shot examples.
    labels = [example["label"] for example in examples]
    most_common_label = max(set(labels), key=labels.count)
    return most_common_label

# Show how different few shot sets change classification behavior.
def run_demo():
    test_message = "Why did my payment fail when I used my new card?"
    original_label = classify_message(test_message, few_shot_examples)

    # Print original classification result using initial few shot examples.
    print("Original label with initial few shots:", original_label)

    # Add new sarcastic style example labeled as billing category.
    updated_examples = few_shot_examples + [
        {"message": "Love being charged twice for nothing, thanks a lot.", "label": "billing"}
    ]

    # Classify same message again using updated few shot examples.
    updated_label = classify_message(test_message, updated_examples)
    print("Updated label with sarcasm example:", updated_label)

    # Show both example sets sizes to emphasize maintainability concept.
    print("Initial examples count:", len(few_shot_examples))
    print("Updated examples count:", len(updated_examples))

run_demo()



### **2.3. Handling uncertain cases**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_02/Lecture_B/image_02_03.jpg?v=1768764862" width="250">



>* Ambiguous inputs can’t always match existing labels
>* Define and encode clear behavior for model uncertainty

>* Add special labels for uncertain or unusual inputs
>* Route uncertain cases to humans or extra processing

>* Use model explanations to detect low confidence
>* Route doubtful cases to humans and refine rules



In [ ]:
#@title Python Code - Handling uncertain cases

# Demonstrate handling uncertain classifications using simple rule based logic.
# Show how a special uncertain label can route inputs for human review.
# Simulate model explanations and detect low confidence using keyword based checks.

# pip install langchain-openai langchain-community.

# Define possible classification labels including uncertain fallback.
labels = ["billing", "technical", "general", "needs_human_review"]

# Define example customer messages with simulated model explanations.
examples = [
    {
        "message": "My internet is down since yesterday evening.",
        "label": "technical",
        "explanation": "Clearly a technical connectivity issue with internet service."
    },
    {
        "message": "I think I was double charged but not totally sure.",
        "label": "billing",
        "explanation": "Possibly billing related but information seems incomplete and uncertain."
    },
    {
        "message": "Your service is kind of weird, maybe broken or maybe just slow.",
        "label": "technical",
        "explanation": "Description is vague, could be normal slowness or actual outage, not clear."
    }
]

# Define keywords that suggest low confidence or missing information.
uncertain_keywords = ["uncertain", "not clear", "possibly", "incomplete", "vague"]

# Define function that upgrades label to human review when explanation seems uncertain.
def route_with_uncertainty(original_label, explanation):
    lowered = explanation.lower()
    for word in uncertain_keywords:
        if word in lowered:
            return "needs_human_review"
    return original_label

# Process each example and print routing decisions with explanations.
for item in examples:
    final_label = route_with_uncertainty(item["label"], item["explanation"])
    print("Message:", item["message"])
    print("Model label:", item["label"], "| Final route:", final_label)
    print("Explanation:", item["explanation"])
    print("-")



## **3. Effective QA Prompting**

### **3.1. Clarifying User Questions**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_02/Lecture_B/image_03_01.jpg?v=1768764887" width="250">



>* Clarify vague or ambiguous questions before answering
>* Evaluate if the model requests missing details

>* Use qualitative reviews to spot ambiguity failures
>* Track simple metrics and refine clarification instructions

>* Balance when to infer versus clarify questions
>* Use reviewer ratings to tune clarification behavior



### **3.2. Controlled Answer Formatting**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_02/Lecture_B/image_03_02.jpg?v=1768764902" width="250">



>* Control answer structure, detail, and style explicitly
>* Consistent formats simplify scanning, comparison, and evaluation

>* Review sample answers for structure, clarity, tone
>* Track simple metrics to measure consistency and improvement

>* Adjust formatting rules based on real user feedback
>* Iterate until answers are consistent and user-aligned



In [ ]:
#@title Python Code - Controlled Answer Formatting

# Demonstrate controlled answer formatting using simple template prompts.
# Compare freeform answers versus structured formatted answers from a fake model.
# Show how structure helps evaluation and consistency for question answering.
# pip install langchain openai tiktoken if external libraries were actually required.

# Define a simple fake model function that returns canned responses.
def fake_model_response(prompt_text, style_label):
    if style_label == "freeform":
        return "I think the best way is probably to add more insulation and maybe check windows. That usually helps keep houses warmer during winter and can lower bills."
    if style_label == "structured":
        return "Direct answer: Add attic insulation and seal window drafts to reduce heat loss.\nExplanation: Extra insulation slows heat escaping through the roof, and sealing window gaps reduces cold air leaks, which together lower heating costs."
    return "Unknown style requested, please check style label and try again."

# Define a user question about reducing winter heating costs in a small house.
user_question = "How can I reduce my winter heating bill in a small house?"

# Build a freeform prompt without any formatting instructions for the fake model.
freeform_prompt = f"Question: {user_question} Answer in any style you like without constraints."

# Build a controlled formatting prompt with explicit structure and style instructions.
structured_prompt = (
    "You are a home energy assistant. "
    "First give one short direct answer sentence labeled 'Direct answer:'. "
    "Then give one short explanation sentence labeled 'Explanation:'. "
    f"Question: {user_question}"
)

# Get the freeform style answer from the fake model using the first prompt.
freeform_answer = fake_model_response(freeform_prompt, "freeform")

# Get the structured style answer from the fake model using the second prompt.
structured_answer = fake_model_response(structured_prompt, "structured")

# Print both prompts and answers to compare formatting and clarity side by side.
print("FREEFORM PROMPT AND ANSWER:\n")
print(freeform_prompt)
print("\nAnswer:")
print(freeform_answer)

# Print a separator line to clearly distinguish the two different prompting approaches.
print("\n" + "-" * 40 + "\n")

# Print the structured prompt and answer showing controlled answer formatting.
print("STRUCTURED PROMPT AND ANSWER:\n")
print(structured_prompt)
print("\nAnswer:")
print(structured_answer)



### **3.3. Grounded Context Citations**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_02/Lecture_B/image_03_03.jpg?v=1768764933" width="250">



>* Answers must cite specific parts of sources
>* Citations reduce hallucinations and enable easy verification

>* Check if each citation truly supports claims
>* Track simple metrics to spot citation problems

>* Refine instructions so citations are specific, accurate
>* Continuously review outputs, crucial in high-stakes domains



In [ ]:
#@title Python Code - Grounded Context Citations

# Demonstrate grounded context citations using simple question answering example.
# Show how answers reference specific supporting context snippets clearly.
# Evaluate citation quality using simple correctness checks and printed feedback.
# pip install langchain llama-cpp-python sentence-transformers transformers.

# Define a small context knowledge base with labeled snippets.
context_snippets = {
    "A": "Warranty covers manufacturing defects for one year from purchase date in most states.",
    "B": "Warranty does not cover damage from misuse, accidents, or unauthorized repairs by users.",
    "C": "Customers must provide original receipt and product serial number for warranty service eligibility.",
}

# Define a user question about warranty coverage details and limitations.
question = "Does the warranty cover accidental damage and what proof is required?"

# Define a simple model answer that includes grounded citation labels.
model_answer = (
    "Accidental damage is not covered by the warranty [B]. "
    "You must provide the original receipt and serial number as proof [C]."
)

# Define expected supporting snippet labels for each important answer statement.
expected_citations = {
    "accidental damage not covered": "B",
    "proof required receipt serial": "C",
}

# Evaluate whether each expected citation label appears correctly inside the answer.
correct_citations = []
for description, label in expected_citations.items():
    is_present = f"[{label}]" in model_answer
    correct_citations.append((description, label, is_present))

# Print the question, answer, and evaluation of citation correctness.
print("Question:", question)
print("\nModel answer with citations:")
print(model_answer)
print("\nCitation evaluation results:")
for description, label, is_present in correct_citations:
    status = "OK" if is_present else "MISSING"
    print(f"Claim '{description}' should cite [{label}]: {status}")

# Print one cited context snippet to show how users can verify grounding.
label_to_show = "B"
print("\nCited context for label B:")
print(f"[{label_to_show}] {context_snippets[label_to_show]}")



# <font color="#418FDE" size="6.5" uppercase>**Task Specific Prompts**</font>


In this lecture, you learned to:
- Design prompts optimized for summarization, classification, and Q&A with Llama 3. 
- Implement LangChain chains that encapsulate task-specific prompts and Llama 3 calls. 
- Evaluate and iterate on prompts based on qualitative and simple quantitative feedback. 

In the next Module (Module 3), we will go over 'Chains And Memory'